## Imports

In [2]:
from midi_to_dataframe import NoteMapper, MidiReader, MidiWriter
import IPython
from IPython.display import Image, IFrame
from PIL import Image
import seaborn as sns
import pandas as pd
import numpy as np
import os
import json
import music21
import pickle

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Load Mappings File

A **NoteMapper** object encapsulates how MIDI note information is converted to text to be displayed within a DataFrame. This object is initialized from a JSON file, containing three objects:

* **midi-to-text**: JSON mapping of MIDI program numbers to their textual representation. Used when converting MIDI files to DataFrames.
    * For example: *{"0": "piano"}*
* **text-to-midi**: JSON mapping of textual representations of MIDI instruments to MIDI program numbers. Used when writing DataFrames to MIDI.
    * For example: *{"piano": 0}*
* **durations**: JSON mapping of textual representations of MIDI instruments to predefined quantization values (in quarter notes). Used when converted MIDI files to DataFrames.
    * For example: *{"piano": [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3, 4, 6, 8, 12, 16]}*

In [3]:
note_mapping_config_path = "./config/map-to-group.json"
note_mapper = NoteMapper(note_mapping_config_path)

## Convert a MIDI file to a DataFrame

The **MidiReader** object is used to read a MIDI file from disk and convert it to a **DataFrame**. A **NoteMapper** object is passed to the MidiReader upon initialization to handle the MIDI to text conversion of note durations and program names.

In [5]:
reader = MidiReader(note_mapper)

# getting midi files
filepath = "./datasets/dataset_pop/"
MidiDataDF = pd.DataFrame()
count = 0

for filename in os.listdir(filepath):
    if filename.endswith(".midi"):
        
        # create file path
        count += 1
        #print(count, filename, end = " ")
        fullFilePath = filepath+filename

        # read file as dataframe
        tempDF = reader.convert_to_dataframe(fullFilePath)
        #print(tempDF.shape[0])
        MidiDataDF = MidiDataDF.append(tempDF)


In [6]:
#MidiDataDF

## MIDI DataFrame

The created DataFrame object contains the sequence of musical notes found in the input MIDI file, quantized by 16th notes and the following rows:

* **timestamp**: the MIDI timestamp (tick)
* **bpm**: the beats per minute at the timestamp
* **time_signature**: the time signature at the timestamp
* **measure**: the measure number at the timestamp
* **beat**: the downbeat within the current measure at the timestamp, in quarter notes
* **notes**: a textual representation of the notes played at the current timestamp

In [7]:
NotesDataDF = MidiDataDF[["notes"]]
#NotesDataDF

## Vocabulary Building

In [8]:
vocabulary = ["rest"] # add rest by default

### Instruments

In [9]:
instruments = ['bass', 'synthlead', 'synthfx', 'reed',
               'percussive', 'organ', 'guitar', 'pipe',
               'soundfx', 'chromatic', 'ethnic', 'piano',
               'brass', 'synthpad', 'ensemble', 'strings']

percussionInstruments = ['acousticbassdrum', 'bassdrum', 'rimshot', 'acousticsnare',
                         'clap', 'snare', 'lowfloortom', 'closedhat', 'highfloortom',
                         'pedalhat', 'lowtom', 'openhat', 'lowmidtom', 'highmidtom',
                         'crashcymbal', 'hightom', 'ridecymbal', 'chinesecymbal',
                         'ridebell', 'tambourine', 'splashcymbal', 'cowbell', 'vibraslap',
                         'highbongo', 'lowbongo', 'mutehighconga', 'openhighconga', 'lowconga',
                         'hightimbale', 'lowtimbale', 'highagogo', 'lowagogo', 'cabasa',
                         'maracas', 'shortwhistle', 'longwhistle', 'shortguiro', 'longguiro',
                         'claves', 'highwoodblock', 'lowwoodblock', 'mutecuica', 'opencuica',
                         'mutetriangle', 'opentriangle']

### Chords

In [10]:
notesTemp = list(NotesDataDF["notes"])
chordsList = []

for i in notesTemp:
    if i != "rest":
        indexSplit = i.split(",")
        for j in indexSplit:
            chord = j.split("_")
            if chord[0] != "percussion":
                chordsList.append(chord[1])
        
chordsList = set(chordsList)
print(chordsList)

{'g7', 'd9', 'c1', 'a1', 'g8', 'c3', 'd7', 'c#2', 'a8', 'b2', 'f6', 'b7', 'f#7', 'g3', 'd2', 'f8', 'g4', 'c#8', 'e5', 'g#7', 'd#4', 'g#4', 'a#8', 'c0', 'a#6', 'a#5', 'c#4', 'b3', 'b4', 'c6', 'f#8', 'e9', 'g#2', 'd6', 'g0', 'b5', 'f#0', 'e1', 'd#5', 'c#3', 'g6', 'f5', 'd#7', 'c8', 'g5', 'd4', 'c#1', 'a6', 'e2', 'd8', 'g#8', 'g#1', 'g9', 'f0', 'f7', 'd5', 'a#0', 'e6', 'f4', 'd#8', 'f#2', 'b8', 'e4', 'f2', 'b1', 'c4', 'g2', 'g#6', 'g1', 'e7', 'c9', 'a2', 'f#1', 'c7', 'g#5', 'a4', 'd1', 'g#3', 'f9', 'e3', 'e0', 'c#0', 'c5', 'b0', 'd3', 'a#2', 'c#5', 'a#1', 'a5', 'a3', 'd#2', 'd#9', 'c#7', 'f#3', 'f#6', 'c#6', 'a#7', 'a7', 'e8', 'f1', 'd#3', 'b6', 'c2', 'f#5', 'c#9', 'd#6', 'd#1', 'f#4', 'd#0', 'a#4', 'a#3', 'f3', 'd0'}


### Durations

In [11]:
f = open(note_mapping_config_path)
jsonData = json.load(f)
f.close()

print(jsonData["durations"])

{'piano': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'chromatic': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'organ': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'guitar': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0, 24.0, 28.0, 32.0], 'bass': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'strings': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'ensemble': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'brass': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'reed': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'pipe': [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0, 8.0, 12.0, 16.0], 'synthlead': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16.0], 'synthpad': [0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.5, 3.0, 4.0, 6.0, 8.0, 12.0, 16

### Build Vocabulary List

In [12]:
for i in instruments:
    for c in chordsList:
        for d in jsonData["durations"][i]:
            word = str(i) + "_" + str(c) + "_" + str(d)
            vocabulary.append(word)
            
for p in percussionInstruments:
    word = "percussion_" + str(p) + "_0.25"
    vocabulary.append(word)
            
print(len(vocabulary))
#print(vocabulary)

21629


### Create Dictionary to Map Word onto Integers

In [13]:
vocabMappings = dict(zip(vocabulary, range(0, len(vocabulary))))
#print(vocabMappings)

### Mapping Data To Integers (Forward Mapping)

In [14]:
notesTemp = list(NotesDataDF["notes"])
mappedNotes = []

for i in notesTemp:
    indexSplit = i.split(",")
    for j in indexSplit:
        if len(indexSplit) > 1:
            mapping = int(vocabMappings[j]) * (-1)
        else:
            mapping = int(vocabMappings[j])
        mappedNotes.append(mapping)

print(len(mappedNotes))
#mappedNotes

1183421


In [17]:
vocabularyChars = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"] # 10 is minus (-) and 11 is comma (,)
mappedNotesChars = []

for note in mappedNotes:
    temp = str(note)
    tempArr = [*temp]
    if tempArr[0] == "-":
        tempArr[0] = "10"
    tempArr.append("11")
    mappedNotesChars.extend(tempArr)
    
# Convert the list to a string with each element separated by a space
mappedNotesString = " ".join(mappedNotesChars)

print(len(mappedNotesString))
print(mappedNotesString[0:100])

16591671
10 1 4 5 8 6 11 10 1 5 2 4 6 11 10 1 4 3 3 1 11 0 11 0 11 0 11 10 1 4 3 3 1 11 10 1 5 2 4 6 11 10 1 


## RNN + LSTM Model

### Preparing Data For Input Into Model

In [41]:
class Data:
    features = []
    targets = []
    featureLength = 1000
    def __init__(self, features, targets, featureLength):
        self.features = features
        self.targets = targets
        self.featureLength = featureLength

In [42]:
print("To NP:")
mappedNotesChars = np.array(mappedNotesChars, dtype=float)

d = Data([],[],1000)

for i in range(len(mappedNotesChars)-d.featureLength):
    #print("Index:", i)
    tempF = mappedNotesChars[i:i+d.featureLength]
    d.features.append(tempF)
    tempT = mappedNotesChars[i+d.featureLength]
    d.targets.append(tempT)
    
n_patterns = len(d.targets)

To NP:


In [43]:
print(len(d.targets))
print(len(d.features[0]))

7201917
1000


In [45]:
print(d.features[0][0])
print(d.targets[0])

10.0
1.0


In [ ]:
pickle.dump(d, open('pop_data.pickle', 'wb'))

In [17]:
features = np.reshape(features, (n_patterns, featureLength, 1))

MemoryError: Unable to allocate 53.7 GiB for an array with shape (7206088, 1000) and data type float64

In [ ]:

targets = np.array(targets)
targets = to_categorical(targets, len(vocabularyChars))

In [ ]:
print(features[0], targets[1])

### Create and Train Model

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(features.shape[1],features.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(vocabulary), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(features, targets, epochs=20, batch_size=128, callbacks=callbacks_list)

### Mapping Integers to Data (Backward Mapping)

In [ ]:
output = mappedNotes[0:2000]
concatStr = ""
reverseMapping = []

for i in output:
    if i < 0:
        i = i * (-1)
        result = [new_k for new_k in vocabMappings.items() if new_k[1] == i][0][0]
        concatStr = concatStr + "," + result
    else:
        if concatStr != "":
            reverseMapping.append(concatStr.lstrip(","))
            concatStr = ""
        result = [new_k for new_k in vocabMappings.items() if new_k[1] == i][0][0]
        reverseMapping.append(result)
        
reverseMapping

## Convert a DataFrame to a MIDI File

The **MidiWriter** object handles writing properly-formatted **DataFrames** as playable MIDI files. A **NoteMapper** object is passed to the MidiWriter upon initialization to handle the text to MIDI conversion of note durations and program names. The path and filename of the output MIDI file is specified in the *convert_to_midi* call of the MidiWriter.

In [ ]:
outputDF = pd.DataFrame(reverseMapping, columns =['notes'])
outputDF["bpm"] = 125

cols = outputDF.columns.tolist()
cols = cols[-1:] + cols[:-1]
outputDF = outputDF[cols]

outputDF

In [ ]:
# Drop the first 15 rows of the dataframe, which represented 1 measure of silence

# Write the modified DataFrame to disk as a playable MIDI file
writer = MidiWriter(note_mapper)
writer.convert_to_midi(outputDF, "./output.midi")

parsed = music21.converter.parse("./output.midi")
parsed.write('musicxml.png', fp='./sheets/Score')
pdfPath = parsed.write('lily.pdf', fp='./sheets/Score')

filepath = "./sheets/"
for filename in os.listdir(filepath):
    if filename.endswith(".png"):
        im = Image.open(filepath+filename)
        bg = Image.new("RGB", im.size, (255,255,255))
        bg.paste(im,im)
        os.remove(filepath+filename)
        filename = filename.replace(".png",".jpg")
        bg.save(filepath+filename)

os.remove(filepath + "Score")
os.remove(filepath + "Score.musicxml")

IFrame(str(pdfPath), width=900, height=800)